In [1]:
import random
import os
import glob
import shutil

In [2]:
random_country_dir = random.choice(os.listdir("../country_data"))
random_country_dir

'SVK_Slovakia'

In [3]:
figures = glob.glob(f"../country_data/{random_country_dir}/figures/*.png", recursive = True)
figures

['../country_data/SVK_Slovakia/figures/SVK_CDIAC_1.png',
 '../country_data/SVK_Slovakia/figures/SVK_IEA_1.png',
 '../country_data/SVK_Slovakia/figures/SVK_CAIT_lucf_vs_nolucf.png',
 '../country_data/SVK_Slovakia/figures/SVK_CAIT_gases_1.png',
 '../country_data/SVK_Slovakia/figures/SVK_BP_1.png',
 '../country_data/SVK_Slovakia/figures/SVK_Minx_top20_subsectors.png',
 '../country_data/SVK_Slovakia/figures/SVK_GCP_1.png',
 '../country_data/SVK_Slovakia/figures/SVK_CO2_totals.png',
 '../country_data/SVK_Slovakia/figures/SVK_UNFCCC_AI_1.png',
 '../country_data/SVK_Slovakia/figures/SVK_EIA_1.png']

In [4]:
random_fig = random.choice(figures)
random_fig

'../country_data/SVK_Slovakia/figures/SVK_BP_1.png'

In [5]:
shutil.copyfile(random_fig, "../supplemental/random_daily_fig.png",)

'../supplemental/random_daily_fig.png'

In [6]:
print("random_fig path imported")

random_fig path imported
